<a href="https://colab.research.google.com/github/LAnGallego/AlertaEnchente/blob/main/AlertaEnchente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

!pip install -q -U google-generativeai



In [12]:
#Configurações iniciais
import google.generativeai as genai
import numpy as np
import pandas as pd

import requests

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')

genai.configure(api_key = api_key)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


title = "A próxima geração de IA para desenvolvedores e google workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e google workspace"
                "\n"
                "Artigo completo"
              "\n"
              "Gemini API Google AI studio: uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")


In [ ]:
request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")


In [29]:


api_key='b6ee2a44f6c9418dba2155146241105'
city = "São Paulo"

#base_url = "http://api.openweathermap.org/data/2.5/weather?"
url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no"

#url = f"{base_url}appid={api_key}&q={city}"
response = requests.get(url)




In [30]:
if response.status_code == 200:
    data = response.json()
    main = data['main']
    temperature = main['temp']
    humidity = main['humidity']
    pressure = main['pressure']
    report = data['weather']
    print(f"Temperatura: {temperature - 273.15:.2f}°C")
    print(f"Umidade: {humidity}%")
    print(f"Pressão: {pressure} hPa")
    print(f"Descrição: {report[0]['description']}")
else:
    print("Erro na requisição.")







KeyError: 'main'

In [37]:
import requests

api_key = 'b6ee2a44f6c9418dba2155146241105'
city = 'Porto Alegre'
url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}&aqi=no"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    temp_c = data['current']['temp_c']
    condition = data['current']['condition']['text']
    prec = data['current']['precip_mm']
    print(f"Temperatura em {city}: {temp_c}°C, {condition}, precipitação de {prec}")


else:
    print("Erro na requisição.")

Temperatura em Porto Alegre: 19.0°C, Light rain, precipitação de 1.67


In [22]:
data


{'location': {'name': 'London',
  'region': 'City of London, Greater London',
  'country': 'United Kingdom',
  'lat': 51.52,
  'lon': -0.11,
  'tz_id': 'Europe/London',
  'localtime_epoch': 1715443228,
  'localtime': '2024-05-11 17:00'},
 'current': {'last_updated_epoch': 1715442300,
  'last_updated': '2024-05-11 16:45',
  'temp_c': 19.0,
  'temp_f': 66.2,
  'is_day': 1,
  'condition': {'text': 'Sunny',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png',
   'code': 1000},
  'wind_mph': 11.9,
  'wind_kph': 19.1,
  'wind_degree': 50,
  'wind_dir': 'NE',
  'pressure_mb': 1020.0,
  'pressure_in': 30.12,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 60,
  'cloud': 0,
  'feelslike_c': 19.0,
  'feelslike_f': 66.2,
  'vis_km': 9.0,
  'vis_miles': 5.0,
  'uv': 5.0,
  'gust_mph': 16.3,
  'gust_kph': 26.3}}